In [1]:
import os

%pwd

'd:\\software_3\\Generative_models\\Text_models\\chat_gpt2\\SFT_LoRA_QLoRA_RLHF'

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\software_3\\Generative_models\\Text_models\\chat_gpt2'

# QLORA Supervised Fine Tunning

This notebook implements the QLORA SFT from this paper [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314) 

The QLORA SFT is an quantized adapter model which means the model weights are quantized into the smaller memeory size
the main advantage of this approach is that while finetunning we does not need to finetune the entire model instead we can only finetune the only the adapter layers resulting into faster training and can work with the consumer hardwares.

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from gpt import GPTModel, MultiHeadAttention, LayerNorm, FeedForward
from model_args import BASE_CONFIG
from utils.load_and_save_models import load_model, save_model
from utils.train import finetune_model, dataloaders
from utils.download_dataset import download_and_load_dataset, partition_data
from utils.generate import generate,
from utils.token_converter import get_tokenizer, text_to_token_ids, token_ids_to_text

## Creating the QLORA GPTMODEL

In [24]:
class QuantizedLinear(nn.Module):
    """
    Quantized Linear layers using 4-bit quantization
    """
    def __init__(self, linear_layer, bits=4):
        super().__init__()
        self.linear = linear_layer
        self.bits = bits
        self.scale = None
        self.zero_point = None

    def quantize_weights(self):
        # quantize weights to 4 bits
        weights = self.linear.weight.data

        # calculate scale and zero point for quantization
        w_min, w_max = weights.min(), weights.max()
        self.scale = (w_max - w_min) / (2**self.bits - 1)
        self.zero_point = w_min

        # Quantize weights
        quantized = torch.round((weights - self.zero_point) / self.scale)
        quantized = torch.clamp(quantized, 0, 2**self.bits - 1)

        # store quantized weights
        self.register_buffer("quantized_weights", quantized)

    def forward(self, x):
        if not hasattr(self, "quantized_weights"):
            self.quantize_weights()

        # Dequantize weights for the forward pass
        dequantized_weights = self.quantized_weights * self.scale + self.zero_point

        # use dequantized weights for computation
        return F.linear(x, dequantized_weights, self.linear.bias)

In [25]:
class QLoRALinear(nn.Module):
    """QLoRA implementation (quantized base layer + lora adapter)"""
    def __init__(self, linear_layer, r=8, lora_alpha=16, lora_dropout=0.1, bits=4):
        super().__init__()
        self.quantized_linear = QuantizedLinear(linear_layer, bits)
        self.r = r
        self.lora_alpha = lora_alpha
        self.lora_dropout = lora_dropout

        self.in_features = linear_layer.in_features
        self.out_features = linear_layer.out_features

        # creating the LoRA adapters (in full dimension)
        self.lora_A = nn.Linear(self.in_features, self.r, bias=False)
        self.lora_B = nn.Linear(self.r, self.out_features, bias=False)

        # initialize the lora weights
        nn.init.kaiming_uniform_(self.lora_A.weight, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B.weight)

        # scaling factor
        self.scaling = self.lora_alpha / self.r

    def forward(self, x):
        # Quantized forward pass
        quantized_output = self.quantized_linear(x)

        # Lora forward pass full precision
        lora_output = self.lora_B(self.lora_A(F.dropout(x, self.lora_dropout, self.training)))

        # combine quantized and lora weights
        return quantized_output + self.scaling * lora_output

In [26]:
class QLoRAMultiHeadAttention(MultiHeadAttention):
    """
    MultiHeadAttention with QLoRA applied to q, k, v projections
    """
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False,
                 r=8, lora_alpha=16, lora_dropout=0.1, bits=4):
        super().__init__(d_in, d_out, context_length, dropout, num_heads, qkv_bias)

        self.W_query = QLoRALinear(self.W_query, r, lora_alpha, lora_dropout, bits)
        self.W_key = QLoRALinear(self.W_key, r, lora_alpha, lora_dropout, bits)
        self.W_value = QLoRALinear(self.W_value, r, lora_alpha, lora_dropout, bits)

In [27]:
class QLoRATransformerBlock(nn.Module):
    """
    Transformer block with QLoRA applied to attention layers
    """
    def __init__(self, cfg, r=8, lora_alpha=16, lora_dropout=0.1, bits=4):
        super().__init__()

        self.att = QLoRAMultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"],
            r=r,
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout,
            bits=bits
        )

        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x

In [28]:
class QLoRAGPTModel(GPTModel):
    """
    GPT model with QLoRA applied to attention layers
    """
    def __init__(self, cfg, r=8, lora_alpha=16, lora_dropout=0.1, bits=4):
        super().__init__(cfg)  # Fixed: was super.__init__(cfg)

        # Replace transformer blocks with QLoRA versions
        self.trf_blocks = nn.Sequential(
            *[QLoRATransformerBlock(cfg, r, lora_alpha, lora_dropout, bits)
            for _ in range(cfg["n_layers"])]
        )

    def get_trainable_parameters(self):
        # get only lora trainable parameters
        trainable_params = []
        for name, param in self.named_parameters():
            if "lora_A" in name or "lora_B" in name:
                trainable_params.append(param)
        return trainable_params

    def print_trainable_parameters(self):
        """Print information about trainable parameters"""
        trainable_params = self.get_trainable_parameters()
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params_count = sum(p.numel() for p in trainable_params)
        
        print(f"Total parameters: {total_params:,}")
        print(f"Trainable parameters (LoRA only): {trainable_params_count:,}")
        print(f"Percentage of trainable parameters: {100 * trainable_params_count / total_params:.2f}%")
        print(f"Memory savings: Base model is quantized to {self.trf_blocks[0].att.W_query.quantized_linear.bits}-bit precision")


In [29]:
def create_qlora_model(base_model_path, r=8, lora_alpha=16, lora_dropout=0.1, bits=4):
    """
    Create a QLoRA-enabled GPT model from a pre-trained model
    """
    qlora_model = QLoRAGPTModel(BASE_CONFIG, r, lora_alpha, lora_dropout, bits)

    print("Loading base model")
    state_dict = torch.load(base_model_path, map_location="cpu")

    # Load weights that don't have LoRA (embedding, norm, output layers)
    model_dict = qlora_model.state_dict()
    for name, param in state_dict.items():
        if name in model_dict and "lora" not in name and "quantized" not in name:
            model_dict[name] = param  # Fixed: was model_dict[param] = param

    qlora_model.load_state_dict(model_dict)
    print("Model weights loaded successfully")

    # Freeze the base model parameters (quantized layers)
    for name, param in qlora_model.named_parameters():
        if "lora" not in name:
            param.requires_grad = False

    return qlora_model

In [30]:
def estimate_memory_usage(model, bits=4):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.get_trainable_parameters())
    
    # Estimate memory usage (rough calculation)
    # Base model: quantized to 4-bit
    base_memory = (total_params - trainable_params) * bits / 8  # bytes
    # LoRA parameters: full precision (32-bit)
    lora_memory = trainable_params * 4  # bytes
    
    total_memory_mb = (base_memory + lora_memory) / (1024 * 1024)
    
    print(f"Estimated memory usage: {total_memory_mb:.2f} MB")
    print(f"  - Base model (quantized): {base_memory / (1024 * 1024):.2f} MB")
    print(f"  - LoRA adapters: {lora_memory / (1024 * 1024):.2f} MB")

In [ ]:
pretrained_model_path = "gpt_models\\GPT2-355M-pretrained.pth"
model = create_qlora_model(pretrained_model_path)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print("####################################")
estimate_memory_usage(model, bits=4)

Loading base model
Model weights loaded successfully
####################################
Estimated memory usage: 79.43 MB
  - Base model (quantized): 77.74 MB
  - LoRA adapters: 1.69 MB


In [38]:
try:
    batch_size, seq_len = 2, 10
    dummy_input = torch.randint(0, BASE_CONFIG["vocab_size"], (batch_size, seq_len)).to(device)
        
    with torch.no_grad():
        output = model(dummy_input)
        
    print(f"Forward pass successful! Output shape: {output.shape}")
        
except Exception as e:
    print(e)

Forward pass successful! Output shape: torch.Size([2, 10, 50257])


In [39]:
data = download_and_load_dataset("instruction-data.json")
tokenizer = get_tokenizer()
device = "cuda" if torch.cuda.is_available() else "cpu"
train_data, test_data, val_data = partition_data(data)
train_dataloader, test_dataloader, val_dataloader = dataloaders(train_data, val_data, test_data, tokenizer=tokenizer,batch_size=16)
optimizer = torch.optim.AdamW(model.get_trainable_parameters(), lr=0.0004, weight_decay=0.1)
num_epochs = 30


Training data length: 935
Test data length: 110
Validation data length: 55


In [40]:
qlora_sft_model = finetune_model(
                 model=model,
                 train_loader=train_dataloader,
                 val_loader=val_dataloader,
                 optimizer=optimizer,
                 device=device,
                 num_epochs=num_epochs,
                 tokenizer=tokenizer,
                 val_data=val_data,
                 eval_freqs=5,
                 eval_iter=5,
                 )

Epoch 1: 100%|██████████| 59/59 [01:19<00:00,  1.34s/it, train_loss=5.152, val_loss=5.065]



[Epoch 1] Sample Generation:
Below is an instruction that describes a task. Write a response that appropriately complates the request.  ## Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'                                                                                                     


Epoch 2: 100%|██████████| 59/59 [01:24<00:00,  1.44s/it, train_loss=4.073, val_loss=4.013]



[Epoch 2] Sample Generation:
Below is an instruction that describes a task. Write a response that appropriately complates the request.  ## Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'                                                                                                     


Epoch 3: 100%|██████████| 59/59 [01:02<00:00,  1.05s/it, train_loss=3.269, val_loss=3.255]


...


[Epoch 28] Sample Generation:
Below is an instruction that describes a task. Write a response that appropriately complates the request.  ## Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'   ### Response: The book. The book. The next to the book. The book. ### Response: The book. The book. The book. The book. The book. The book. The book. The book. The book. The book. The 'I amigo. The 'I amigo. The 'I amigo. The 'I amigo. The 'I amigo. 


Epoch 29: 100%|██████████| 59/59 [00:56<00:00,  1.04it/s, train_loss=1.630, val_loss=1.750]



[Epoch 29] Sample Generation:
Below is an instruction that describes a task. Write a response that appropriately complates the request.  ## Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'   ### Response: The book. He was a good. The book. The next to the book. ###A lot. The book. The book. The book. The book. The book. The book. The book. The book. The book. The book. The book. The day. The 'I amigo. The day. The day. The day. The day. 


Epoch 30: 100%|██████████| 59/59 [01:06<00:00,  1.12s/it, train_loss=1.612, val_loss=1.739]



[Epoch 30] Sample Generation:
Below is an instruction that describes a task. Write a response that appropriately complates the request.  ## Instruction: Convert the active sentence to passive: 'The chef cooks the meal every day.'   ### Response: The book. The book. He is 'I am I am I am I am I am I am I amigo. ###A. The book. The book. The book. The book. The next to the day. The 'I amigo. The 'I amigo. The 'I amigo. The 's. The day. The day. The day. The day
Model training has been completed.
training completed in 38.59 minutes.


In [ ]:
save_model(qlora_sft_model, "instruct-QLORA-GPT2-355M.pth", optimizer=optimizer)

Pretrained model has been saved successfully at d:\software_3\Generative_models\Text_models\chat_gpt2\gpt_models\instruction_finetunned_QLoRA_model.pth


In [42]:
text = "Convert 45 kilometers to meters."

encoded_text =  text_to_token_ids(text, tokenizer).to(device)

idx = encoded_text
token_ids = generate(
        model=qlora_sft_model,
        idx=encoded_text,
        max_new_tokens=30,
        context_size=BASE_CONFIG["context_length"],
        temperature=0.0,
        top_k=None,
        eos_id=None
    )

print(token_ids_to_text(token_ids, tokenizer))

Convert 45 kilometers to meters.

###
The first, and the following the following the following the following the following the following the following the following the sun.



